# Simple LangGraph Agent: From LLMs to Agents

## 1. LLMs vs Agents: Key Differences 🤔

### Traditional LLMs
- **Input → Output**: Simple text-in, text-out pattern
- **Static**: No ability to interact with external systems
- **Single Turn**: One response per query
- **Limited Context**: Only what's in the prompt
- **No Tools**: Cannot perform actual calculations or actions

### AI Agents
- **Interactive**: Can use tools and external APIs
- **Multi-step**: Break down complex tasks into steps
- **Memory**: Maintain state across interactions
- **Planning**: Can reason about next actions
- **Error Handling**: Can retry and recover from failures
- **Tool Usage**: Can perform real actions (calculations, API calls, etc.)

### Visual Comparison

```
LLM Pattern:
User: "What is 15 × 23?"
LLM: "15 × 23 equals 345" (might be wrong!)

Agent Pattern:
User: "What is 15 × 23?"
Agent: 1. Detects math needed
        2. Uses calculator tool  
        3. Returns: "The answer is: 345" (always correct!)
```

## This Demo Shows:
- **Chat Mode**: "How are you?" → Normal conversation
- **Tool Mode**: "What is 5 + 3?" → Uses calculator tool
- **Smart Routing**: Automatically detects what type of response needed

In [ ]:
# Install required packages
!pip install langgraph langchain requests python-dotenv

In [ ]:
# Simple LLM Setup
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from typing import Dict, Any
import os
import requests
from dotenv import load_dotenv

load_dotenv()

# Simple E2E LLM Class
class SimpleLLM:
    def __init__(self):
        self.api_key = os.getenv("E2E_API_KEY")
        self.base_url = os.getenv("E2E_BASE_URL") + "chat/completions"
        
    def call(self, prompt: str) -> str:
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
        payload = {"messages": [{"role": "user", "content": prompt}], "max_tokens": 200}
        
        try:
            response = requests.post(self.base_url, headers=headers, json=payload, timeout=10)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return "Sorry, I'm having trouble right now."
        except:
            return "Sorry, I'm having trouble right now."

llm = SimpleLLM()
print("✅ Simple LLM ready")

In [ ]:
# Simple Calculator Tool
@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions like 5+3, 10*2, etc."""
    try:
        result = eval(expression)
        return str(result)
    except:
        return "Error in calculation"

print("🧮 Calculator tool created!")
print("🎯 This is what makes it an AGENT - it can use TOOLS!")

In [ ]:
# Simple Agent Functions
import re

def create_state(user_input: str) -> Dict[str, Any]:
    return {"user_input": user_input, "needs_math": False, "result": ""}

def check_math_need(state: Dict) -> Dict:
    # Simple check: if it has numbers and math words, it's math
    text = state["user_input"].lower()
    has_numbers = bool(re.search(r'\d', text))
    has_math_words = any(word in text for word in ["calculate", "what is", "+", "-", "*", "/", "plus", "minus", "times", "divided"])
    
    state["needs_math"] = has_numbers and has_math_words
    return state

def do_math(state: Dict) -> Dict:
    # Extract math expression better
    text = state["user_input"]
    
    # Try different patterns to find math
    patterns = [
        r'what is\s+([0-9+\-*/\s]+)',  # "what is 5 + 3"
        r'calculate\s+([0-9+\-*/\s]+)',  # "calculate 10 * 2"
        r'([0-9]+\s*[+\-*/]\s*[0-9]+)',  # "5 + 3" anywhere
    ]
    
    expression = None
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            expression = match.group(1).strip().replace(" ", "")
            break
    
    if expression:
        result = calculator.invoke({"expression": expression})
        state["result"] = f"🔧 Calculator: {expression} = {result}"
    else:
        state["result"] = "Could not find math expression"
    
    return state

def chat_response(state: Dict) -> Dict:
    # Use LLM for general chat
    response = llm.call(f"Please respond to: {state['user_input']}")
    state["result"] = f"💬 Chat: {response}"
    return state

def route_request(state: Dict) -> str:
    return "do_math" if state["needs_math"] else "chat_response"

print("✅ Simple agent functions ready")

In [ ]:
# Create the Agent
workflow = StateGraph(dict)

# Add nodes
workflow.add_node("check_math", check_math_need)
workflow.add_node("do_math", do_math)
workflow.add_node("chat_response", chat_response)

# Set entry point
workflow.set_entry_point("check_math")

# Add routing
workflow.add_conditional_edges(
    "check_math",
    route_request,
    {
        "do_math": "do_math",
        "chat_response": "chat_response"
    }
)

# End points
workflow.add_edge("do_math", END)
workflow.add_edge("chat_response", END)

# Compile the agent
agent = workflow.compile()

print("✅ Simple LangGraph Agent created!")
print("🔄 Flow: Input → Check Math → Route → Tool/Chat → Result")

In [ ]:
# Test Examples
def test_agent(question: str):
    print(f"👤 User: {question}")
    state = create_state(question)
    result = agent.invoke(state)
    print(f"🤖 {result['result']}")
    print("-" * 40)

print("🧮 MATH EXAMPLES:")
test_agent("What is 5 + 3?")
test_agent("Calculate 10 * 2")
test_agent("What is 15 - 7?")

print("\n💬 CHAT EXAMPLES:")
test_agent("Hello!")
test_agent("How are you?")
test_agent("Who are you?")

In [ ]:
# Interactive Demo
print("🎮 Simple Agent Demo")
print("Math: 'What is 5 + 3?' | Chat: 'Hello!'")
print("Type 'quit' to exit")

while True:
    user_input = input("\n👤 You: ")
    if user_input.lower() == 'quit':
        break
    
    state = create_state(user_input)
    result = agent.invoke(state)
    print(f"🤖 {result['result']}")

In [ ]:
# Summary
print("✅ Simple Agent Complete!")
print()
print("🔄 How it works:")
print("1. Text input → LLM response")
print("2. Math input → Calculator tool") 
print()
print("📝 Examples:")
print("• 'Hello' → LLM generates chat response")
print("• 'What is 5+3?' → Uses calculator tool = 8")

## How It Works - The Agent Flow

### 🔍 Step-by-Step Execution:

1. **Input Analysis**: Checks if the question needs math calculation
2. **Smart Routing**: Routes to either math calculator or chat response  
3. **Processing**: 
   - **Math Path**: Uses calculator tool
   - **Chat Path**: Uses simple responses or E2E Networks API
4. **Response**: Returns the result

### 🎯 Key Agent Capabilities Demonstrated:

✅ **Tool Usage** - Can perform actual calculations  
✅ **Decision Making** - Routes based on input type  
✅ **State Management** - Tracks conversation context  
✅ **Multi-modal Response** - Handles both chat and computation  

### 📈 Why This Matters:

- **LLM**: "What's 15 * 23?" → "15 * 23 = 345" (might be wrong!)
- **Agent**: "What's 15 * 23?" → Uses calculator → "The answer is: 345" (always correct!)

**Simple and Clean!** 🎯